In [ ]:
!pip install eccodes herbie-data ecmwflibs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp drive/MyDrive/preamble.pkl preamble.pkl # copy the preamble file

In [ ]:
!mkdir /root/data
!mkdir /root/data/hrrr

In [ ]:
!tar -xf drive/MyDrive/hrrr16.tgz -C / # extracting the VIL GRIB files

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import pickle as pkl

In [ ]:
with open('preamble.pkl', 'rb') as f:
    preamble = pkl.load(f)
    print('Preamble file loaded')

# Coordinate of KDFW
KDFW = [-96.973496106, 32.584330996] # lon, lat

# WXR region of interest, check the _plotting_and_roi.ipynb for details
# WX_ROI = [(-103, -90.5), (26.5, 38)] # lon, lat
WX_ROI = [(KDFW[0] - 3, KDFW[0] + 3), (KDFW[1] - 3, KDFW[1] + 3)] # lon, lat

Preamble file loaded


In [ ]:
from datetime import datetime, timedelta

def print_dates_between(start_date, end_date, time_suffix): # 1 day addition
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")

    current = start
    while current <= end:
        yield(current.strftime("%Y-%m-%d") + " " + time_suffix)
        current += timedelta(days=1)

In [ ]:
from_datetime = list(print_dates_between("2016-01-01", "2016-12-31", "00:00"))
to_datetime = list(print_dates_between("2016-01-01", "2016-12-31", "23:45"))
print('Roll call created')
print(from_datetime)
print(to_datetime)
print('There are {} days'.format(len(from_datetime)))

Roll call created
['2016-01-01 00:00', '2016-01-02 00:00', '2016-01-03 00:00', '2016-01-04 00:00', '2016-01-05 00:00', '2016-01-06 00:00', '2016-01-07 00:00', '2016-01-08 00:00', '2016-01-09 00:00', '2016-01-10 00:00', '2016-01-11 00:00', '2016-01-12 00:00', '2016-01-13 00:00', '2016-01-14 00:00', '2016-01-15 00:00', '2016-01-16 00:00', '2016-01-17 00:00', '2016-01-18 00:00', '2016-01-19 00:00', '2016-01-20 00:00', '2016-01-21 00:00', '2016-01-22 00:00', '2016-01-23 00:00', '2016-01-24 00:00', '2016-01-25 00:00', '2016-01-26 00:00', '2016-01-27 00:00', '2016-01-28 00:00', '2016-01-29 00:00', '2016-01-30 00:00', '2016-01-31 00:00', '2016-02-01 00:00', '2016-02-02 00:00', '2016-02-03 00:00', '2016-02-04 00:00', '2016-02-05 00:00', '2016-02-06 00:00', '2016-02-07 00:00', '2016-02-08 00:00', '2016-02-09 00:00', '2016-02-10 00:00', '2016-02-11 00:00', '2016-02-12 00:00', '2016-02-13 00:00', '2016-02-14 00:00', '2016-02-15 00:00', '2016-02-16 00:00', '2016-02-17 00:00', '2016-02-18 00:00', '

In [ ]:
from herbie import Herbie
import numpy as np
import pandas as pd
import xarray as xr
from tqdm import tqdm

xr.set_options(display_expand_data=False)

In [ ]:
def bin_nxm_rectangles_to_sum(array, n, m, lats, lons, verbose = False):
    # Check if the array dimensions are divisible by n and m
    if array.shape[0] % n != 0 or array.shape[1] % m != 0:
        # Pad the array so that the shape is divisible by n x m
        array = np.pad(array, ((0, n - array.shape[0] % n), (0, m - array.shape[1] % m)))
        lats = np.pad(lats, (0, n - lats.shape[0] % n), constant_values=9999)
        lons = np.pad(lons, (0, m - lons.shape[0] % m), constant_values=9999)

        if verbose:
          print('Original shape: ', array.shape)
          print('Padded shape: ', array.shape)

    # Reshape the array to a new 4D array and then sum over the last two axes to bin the elements
    binned_array = array.reshape(array.shape[0]//n, n, array.shape[1]//m, m).sum(axis=(1,3))
    binned_lats = lats.reshape(lats.shape[0]//n, n).min(axis=1)
    binned_lons = lons.reshape(lons.shape[0]//m, m).min(axis=1)
    return binned_array, binned_lats, binned_lons


In [ ]:
def process_vil(from_datetime, to_datetime, preamble, vil_threshold = 3, verbose = True):

  # Begin downloading with slicing of 30 minutes
  DATES = pd.date_range(
    start=from_datetime, # begin date
    end=to_datetime, # end date
    freq="30min", # frequency: 30 minutes
  )

  vil_arrays = None
  date_array = []
  id_array = []
  roi_max = []

  id = 0
  for date in DATES: # iterate through dates
    try:
      H = Herbie(
      date,  # model run date
      model="hrrr",  # model name
      product="sfc",  # model produce name (model dependent)
      fxx=0,  # forecast lead time
      verbose = False
      )

      # Extract the xarray VIL from the Herbie object
      ds = H.xarray(":VIL:")
      vil = ds.veril # VIL xarray DataArray
    except Exception as e:
      print("Date {}: data corrupted? Will skip this.".format(date))
      print(e)
      continue

    # Forming the mask of the ROI
    roi_lon = WX_ROI[0]
    roi_lat = WX_ROI[1]
    mask_lon = np.logical_and(vil['longitude'].values - 360 > roi_lon[0], vil['longitude'].values - 360 < roi_lon[1])
    mask_lat = np.logical_and(vil['latitude'].values > roi_lat[0], vil['latitude'].values < roi_lat[1])
    mask = np.logical_and(mask_lon, mask_lat)

    # First row index of "True" in mask
    mask_lat_id = np.min(np.where(mask == True)[0])

    # First column index of "True" in mask
    mask_lon_id = np.min(np.where(mask == True)[1])

    # Last row index of "True" in mask
    mask_lat_id_last = np.max(np.where(mask == True)[0])

    # Last column index of "True" in mask
    mask_lon_id_last = np.max(np.where(mask == True)[1])
    preamble_roi_id = (mask_lat_id, mask_lat_id_last), (mask_lon_id, mask_lon_id_last)

    if verbose:
      print('First row index of "True" in mask: ', mask_lat_id)
      print('First column index of "True" in mask: ', mask_lon_id)
      print('Last row index of "True" in mask: ', mask_lat_id_last)
      print('Last column index of "True" in mask: ', mask_lon_id_last)

    # Latitude and longitude values of the ROI
    lat_roi = np.where(mask, vil['latitude'].values, np.nan)
    lon_roi = np.where(mask, vil['longitude'].values, np.nan)
    lat_roi_cropped = lat_roi[mask_lat_id:mask_lat_id_last, mask_lon_id:mask_lon_id_last]
    lon_roi_cropped = lon_roi[mask_lat_id:mask_lat_id_last, mask_lon_id:mask_lon_id_last]

    # Zeroing out VIL values outside ROI, then crop the array to the ROI shape
    vilv = vil.values
    vil_roignee = np.where(preamble['mask'], vilv, 0)
    vil_roignee_cropped = vil_roignee[preamble_roi_id[0][0]:preamble_roi_id[0][1], preamble_roi_id[1][0]:preamble_roi_id[1][1]]
    if verbose:
      print('Cropped shape: ', vil_roignee_cropped.shape)

    if np.max(vil_roignee_cropped) < vil_threshold:
      if verbose:
        print('Skip this date because VIL < vil_threshold {} (VIL is {})'.format(vil_threshold, np.max(vil_roignee_cropped)))
      continue

    lat_roignee_cropped = preamble['latitude'][preamble_roi_id[0][0]:preamble_roi_id[0][1]]
    lon_roignee_cropped = preamble['longitude'][preamble_roi_id[1][0]:preamble_roi_id[1][1]]
    if verbose:
      print('Cropped lat shape: ', lat_roignee_cropped.shape) # contains 1D array of latitudes for vil_roignee_cropped
      print('Cropped lon shape: ', lon_roignee_cropped.shape) # contains 1D array of longitudes for vil_roignee_cropped

    # Compute the statistics of the VIL values in the ROI
    roi_max = np.max(vil_roignee_cropped)
    roi_qs = [np.sum(vil_roignee_cropped > roi_max * i/10) for i in range(1, 10)]

    if verbose:
      print('ROI MAX: ', roi_max)
      print('ROI QS: ', roi_qs)

    # Downsample the ROI resolution
    vil_roignee_cropped_ds, lats_roignee_cropped_ds, lons_roignee_cropped_ds = bin_nxm_rectangles_to_sum(vil_roignee_cropped, 9, 8, lat_roignee_cropped, lon_roignee_cropped)
    if verbose:
      print('Shape of the downsampled VIL data: ', vil_roignee_cropped_ds.shape)
      print('Input layer size ', vil_roignee_cropped_ds.shape[0] * vil_roignee_cropped_ds.shape[1])

    # Append the preprocessed VIL array to the vil_arrays
    if vil_arrays is None:
      vil_arrays = vil_roignee_cropped_ds.reshape(1, vil_roignee_cropped_ds.shape[0], vil_roignee_cropped_ds.shape[1])
    else:
      vil_arrays = np.concatenate((vil_arrays, vil_roignee_cropped_ds.reshape(1, vil_roignee_cropped_ds.shape[0], vil_roignee_cropped_ds.shape[1])))

    date_array.append(date.strftime('%Y-%m-%d %X'))
    id_array.append(id)
    id += 1

  return vil_arrays, date_array, id_array



In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# DOWNLOAD SCRIPT
try:
  big_vil = None
  big_date = []
  big_id = []
  for i, begin_datetime in enumerate(tqdm(from_datetime)):
    end_datetime = to_datetime[i]
    last_processed_datetime = begin_datetime

    vil_arrays, date_array, id_array = process_vil(begin_datetime, end_datetime, preamble,
                                                   vil_threshold = 2.5, verbose = False)
    if vil_arrays is not None:
      if big_vil is None:
        big_vil = vil_arrays
      else:
        big_vil = np.concatenate((big_vil, vil_arrays))
        print("Found {} entries. ".format(big_vil.shape[0]))

    big_date = big_date + date_array
    big_id = big_id + id_array

except Exception as e:
  print('Error occurred')
  print(e)

  0%|          | 1/366 [00:58<5:58:42, 58.97s/it]

Date 2016-01-02 12:00:00: data corrupted? Will skip this.

No index file was found for None
Download the full file first (with `H.download()`).
You will need to remake the Herbie object (H = `Herbie()`)
or delete this cached property: `del H.index_as_dataframe()`
Date 2016-01-02 12:30:00: data corrupted? Will skip this.

No index file was found for None
Download the full file first (with `H.download()`).
You will need to remake the Herbie object (H = `Herbie()`)
or delete this cached property: `del H.index_as_dataframe()`


  1%|          | 3/366 [02:37<5:08:28, 50.99s/it]

Date 2016-01-04 09:00:00: data corrupted? Will skip this.

No index file was found for None
Download the full file first (with `H.download()`).
You will need to remake the Herbie object (H = `Herbie()`)
or delete this cached property: `del H.index_as_dataframe()`
Date 2016-01-04 09:30:00: data corrupted? Will skip this.

No index file was found for None
Download the full file first (with `H.download()`).
You will need to remake the Herbie object (H = `Herbie()`)
or delete this cached property: `del H.index_as_dataframe()`
Date 2016-01-04 14:00:00: data corrupted? Will skip this.

No index file was found for None
Download the full file first (with `H.download()`).
You will need to remake the Herbie object (H = `Herbie()`)
or delete this cached property: `del H.index_as_dataframe()`
Date 2016-01-04 14:30:00: data corrupted? Will skip this.

No index file was found for None
Download the full file first (with `H.download()`).
You will need to remake the Herbie object (H = `Herbie()`)
or de

  2%|▏         | 7/366 [05:57<4:53:02, 48.98s/it]

...

Date 2016-12-30 02:00:00: data corrupted? Will skip this.

No index file was found for None
Download the full file first (with `H.download()`).
You will need to remake the Herbie object (H = `Herbie()`)
or delete this cached property: `del H.index_as_dataframe()`
Date 2016-12-30 02:30:00: data corrupted? Will skip this.

No index file was found for None
Download the full file first (with `H.download()`).
You will need to remake the Herbie object (H = `Herbie()`)
or delete this cached property: `del H.index_as_dataframe()`
Date 2016-12-30 09:00:00: data corrupted? Will skip this.

No index file was found for None
Download the full file first (with `H.download()`).
You will need to remake the Herbie object (H = `Herbie()`)
or delete this cached property: `del H.index_as_dataframe()`
Date 2016-12-30 09:30:00: data corrupted? Will skip this.

No index file was found for None
Download the full file first (with `H.download()`).
You will need to remake the Herbie object (H = `Herbie()`)
or de

100%|█████████▉| 365/366 [7:05:26<01:10, 70.36s/it]

Date 2016-12-31 12:00:00: data corrupted? Will skip this.

No index file was found for None
Download the full file first (with `H.download()`).
You will need to remake the Herbie object (H = `Herbie()`)
or delete this cached property: `del H.index_as_dataframe()`
Date 2016-12-31 12:30:00: data corrupted? Will skip this.

No index file was found for None
Download the full file first (with `H.download()`).
You will need to remake the Herbie object (H = `Herbie()`)
or delete this cached property: `del H.index_as_dataframe()`
Date 2016-12-31 16:00:00: data corrupted? Will skip this.

No index file was found for None
Download the full file first (with `H.download()`).
You will need to remake the Herbie object (H = `Herbie()`)
or delete this cached property: `del H.index_as_dataframe()`
Date 2016-12-31 16:30:00: data corrupted? Will skip this.

No index file was found for None
Download the full file first (with `H.download()`).
You will need to remake the Herbie object (H = `Herbie()`)
or de

100%|██████████| 366/366 [7:06:37<00:00, 69.94s/it]

Found 4640 entries. 


In [ ]:
# vil_arrays, date_array, id_array

In [ ]:
darr = xr.DataArray(big_vil, dims=['id','lat','lon'], attrs={
    'date': big_date,
    'id': big_id
})

In [ ]:
dset = darr.to_dataset(name="VIL")

In [ ]:
!pip install zarr

In [ ]:
dset.to_zarr("hrrr16x.zarr")

In [ ]:
!tar -czvf hrrr16x.tgz hrrr16x.zarr/

hrrr16x.zarr/
hrrr16x.zarr/VIL/
hrrr16x.zarr/VIL/1.0.0
hrrr16x.zarr/VIL/3.1.0
hrrr16x.zarr/VIL/0.1.1
hrrr16x.zarr/VIL/2.0.1
hrrr16x.zarr/VIL/.zarray
hrrr16x.zarr/VIL/1.0.1
hrrr16x.zarr/VIL/2.1.1
hrrr16x.zarr/VIL/3.0.0
hrrr16x.zarr/VIL/3.1.1
hrrr16x.zarr/VIL/0.0.1
hrrr16x.zarr/VIL/1.1.1
hrrr16x.zarr/VIL/2.0.0
hrrr16x.zarr/VIL/3.0.1
hrrr16x.zarr/VIL/.zattrs
hrrr16x.zarr/VIL/0.0.0
hrrr16x.zarr/VIL/1.1.0
hrrr16x.zarr/VIL/2.1.0
hrrr16x.zarr/VIL/0.1.0
hrrr16x.zarr/.zgroup
hrrr16x.zarr/.zattrs
hrrr16x.zarr/.zmetadata


In [ ]:
!cp hrrr16x.tgz drive/MyDrive/hrrr16x.tgz

In [ ]:
# CAUTION: Remove files

In [ ]:
#!rm -rf hrrr16x.zarr

In [ ]:
#!rm -rf test.zarr

In [ ]:
#!rm -rf hrrr16x.tgz

In [ ]:
# Test the created dataset
#!cp drive/MyDrive/hrrr16x.tgz hrrr16y.tgz

In [ ]:
#!tar -xzvf hrrr16y.tgz

hrrr16x.zarr/
hrrr16x.zarr/VIL/
hrrr16x.zarr/VIL/1.0.0
hrrr16x.zarr/VIL/3.1.0
hrrr16x.zarr/VIL/0.1.1
hrrr16x.zarr/VIL/2.0.1
hrrr16x.zarr/VIL/.zarray
hrrr16x.zarr/VIL/1.0.1
hrrr16x.zarr/VIL/2.1.1
hrrr16x.zarr/VIL/3.0.0
hrrr16x.zarr/VIL/3.1.1
hrrr16x.zarr/VIL/0.0.1
hrrr16x.zarr/VIL/1.1.1
hrrr16x.zarr/VIL/2.0.0
hrrr16x.zarr/VIL/3.0.1
hrrr16x.zarr/VIL/.zattrs
hrrr16x.zarr/VIL/0.0.0
hrrr16x.zarr/VIL/1.1.0
hrrr16x.zarr/VIL/2.1.0
hrrr16x.zarr/VIL/0.1.0
hrrr16x.zarr/.zgroup
hrrr16x.zarr/.zattrs
hrrr16x.zarr/.zmetadata


In [ ]:
#dsety = xr.open_zarr('hrrr16x.zarr')